In [6]:
import sqlite3
import pandas as pd
cnx = sqlite3.connect('database.sqlite')

In [7]:
team = pd.read_sql_query("SELECT team_api_id, team_long_name as team FROM Team", cnx)
match = pd.read_sql_query("SELECT * FROM Match", cnx)
team_attr = pd.read_sql_query("SELECT * FROM Team_Attributes", cnx)
match['date'] = match['season'].str.slice_replace(start=4, repl='')

In [8]:
team_dict = dict(zip(team['team_api_id'], team['team']))

In [9]:
match['home_team'] = [ team_dict[id_] for id_ in match['home_team_api_id'] ]
match['away_team'] = [ team_dict[id_] for id_ in match['away_team_api_id'] ]

match['home_win'] = (match['home_team_goal'] >  match['away_team_goal'])
match['tie'] = (match['home_team_goal'] ==  match['away_team_goal'])
match['away_win'] = (match['home_team_goal'] <  match['away_team_goal'])

home_match = match[['date', 'home_team', 'home_win', 'tie', 'away_win', 'home_team_goal', 'away_team_goal']] 
home_match.columns = ['date', 'team', 'wins', 'ties', 'loses', 'goals_scored', 'goals_given']
away_match = match[['date', 'away_team', 'away_win', 'tie', 'home_win', 'away_team_goal', 'home_team_goal']]
away_match.columns = home_match.columns
home_match = home_match.groupby(by=["team", 'date'], as_index=False).sum()
away_match = away_match.groupby(by=["team", 'date'], as_index=False).sum()

In [10]:
df = home_match.merge(away_match, on=["team", 'date'])
new_labels = []
for col in df.columns:
    if '_x' in col:
        new_labels.append('home_' + col[:-2])  
    elif '_y' in col:
        new_labels.append('away_' + col[:-2]) 
    else :
        new_labels.append(col)
df.columns = new_labels
df['wins'] = df['home_wins'] + df['away_wins']
df['loses'] = df['home_loses'] + df['away_loses']
df['ties'] = df['home_ties'] + df['away_ties']
df['goals_scored'] = df['home_goals_scored'] + df['away_goals_scored']
df['goals_given'] = df['home_goals_given'] + df['away_goals_given']

#df['win_pct'] = df['wins'] / (df['wins'] + df['loses'] + df['ties'])
df['games'] = (df['wins'] + df['loses'] + df['ties'])
df['win_pct'] = df['wins'] / df['games']
df['loss_pct'] = df['loses'] / df['games']
df['ties_pct'] = df['loses'] / df['games']
df.head()

,team,date,home_wins,home_ties,home_loses,home_goals_scored,home_goals_given,away_wins,away_ties,away_loses,...,away_goals_given,wins,loses,ties,goals_scored,goals_given,games,win_pct,loss_pct,ties_pct
0,1. FC Kaiserslautern,2010,6,6,5,25,19,7,1,9,...,32,13,14,7,48,51,34,0.382353,0.411765,0.411765
1,1. FC Kaiserslautern,2011,2,5,10,12,28,2,6,9,...,26,4,19,11,24,54,34,0.117647,0.558824,0.558824
2,1. FC Köln,2008,4,5,8,14,25,7,1,9,...,25,11,17,6,35,50,34,0.323529,0.500000,0.500000
3,1. FC Köln,2009,3,6,8,18,29,6,5,6,...,13,9,14,11,33,42,34,0.264706,0.411765,0.411765
4,1. FC Köln,2010,11,2,4,30,21,2,3,12,...,41,13,16,5,47,62,34,0.382353,0.470588,0.470588


In [11]:
team_attr.drop(columns=['buildUpPlayDribbling'], inplace=True)

In [13]:
team_attr.dtypes

id                                 int64
team_fifa_api_id                   int64
team_api_id                        int64
date                              object
buildUpPlaySpeed                   int64
buildUpPlaySpeedClass             object
buildUpPlayDribblingClass         object
buildUpPlayPassing                 int64
buildUpPlayPassingClass           object
buildUpPlayPositioningClass       object
chanceCreationPassing              int64
chanceCreationPassingClass        object
chanceCreationCrossing             int64
chanceCreationCrossingClass       object
chanceCreationShooting             int64
chanceCreationShootingClass       object
chanceCreationPositioningClass    object
defencePressure                    int64
defencePressureClass              object
defenceAggression                  int64
defenceAggressionClass            object
defenceTeamWidth                   int64
defenceTeamWidthClass             object
defenceDefenderLineClass          object
dtype: object

In [8]:
a
t = team_attr.copy()
team_attr_str = t.select_dtypes(include=object).drop(columns=['date', 'team'])
t.drop(columns=team_attr_str.columns, inplace=True)
team_attr_str = pd.get_dummies(team_attr_str, dummy_na=False)
team_attr = pd.concat([t, team_attr_str], axis=1)

In [9]:
team_attr = team_attr.groupby(['team', 'date'], as_index=False).mean()

In [10]:
df = df.merge(team_attr, on=['team', 'date'])

In [13]:
df.drop(columns=['home_wins', 'home_ties', 'home_loses',
       'home_goals_scored', 'home_goals_given', 'away_wins', 'away_ties',
       'away_loses', 'away_goals_scored', 'away_goals_given', 'wins', 'loses',
       'ties', 'games', 'id', 'team_fifa_api_id', 'team_api_id'], inplace=True)

KeyError: "['home_wins' 'home_ties' 'home_loses' 'home_goals_scored'\n 'home_goals_given' 'away_wins' 'away_ties' 'away_loses'\n 'away_goals_scored' 'away_goals_given' 'wins' 'loses' 'ties' 'games' 'id'] not found in axis"

In [19]:
df.drop(columns=['date', 'team']).corr()[['goals_scored', 'goals_given', 'win_pct', 'loss_pct', 'ties_pct']].iloc[7:].sort_values(by=['goals_scored', 'goals_given', 'win_pct', 'loss_pct', 'ties_pct'])

,goals_scored,goals_given,win_pct,loss_pct,ties_pct
chanceCreationPositioningClass_Organised,-0.282573,0.189312,-0.259520,0.226709,0.226709
buildUpPlayPositioningClass_Organised,-0.251707,0.125032,-0.219810,0.169935,0.169935
chanceCreationShootingClass_Normal,-0.175948,0.132661,-0.174070,0.155170,0.155170
buildUpPlayPassing,-0.155370,0.145097,-0.160176,0.153581,0.153581
buildUpPlayPassingClass_Long,-0.091839,0.077298,-0.100266,0.093778,0.093778
chanceCreationPassingClass_Normal,-0.077778,0.034421,-0.075576,0.055989,0.055989
defencePressureClass_Deep,-0.077744,-0.015083,-0.063177,0.037588,0.037588
buildUpPlayPassingClass_Mixed,-0.059866,0.029452,-0.051928,0.040351,0.040351
chanceCreationCrossingClass_Normal,-0.046192,0.024568,-0.038050,0.021860,0.021860
buildUpPlaySpeedClass_Balanced,-0.045318,0.080784,-0.071094,0.075352,0.075352
